In [80]:
## Movie Critics (Meta Critic) Scrapes
## Brady Fowler
## dbf5sd
## CS Class, Masters of Data Science
## Group Members: Matt, Jordan, Andrew

In [81]:
import requests
from bs4 import BeautifulSoup
import time

In [82]:
def read_page(url):
    header_info = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)'}
    # see http://www.whoishostingthis.com/tools/user-agent/ for header info
    response = requests.get(url,headers=header_info)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [94]:
def extract_movie_deets(soup, current_url):
    try:
        critic_rating_val = soup.find(itemprop="ratingValue").get_text().strip()
    except:
        critic_rating_val = 'Unknown'
    try:
        critic_rating_num = soup.find(itemprop="reviewCount").get_text().strip()
    except:
        critic_rating_num = 'Unknown'
    try:
        user_rating = soup.find('a', {'class' : 'metascore_anchor', 'href' : current_url + '/user-reviews'}).get_text().strip()
    except:
        user_rating = 'Unknown'
    try:
        movie_date = soup.find('span', {'class' : 'data', 'itemprop' : 'datePublished'}).get_text().strip()
    except:
        movie_date = 'Unknown'
    try:
        movie_name = soup.find('span', {'itemprop' : 'name'}).get_text().strip()
    except:
        movie_name = 'Unknown'
    return [critic_rating_val, critic_rating_num, user_rating, movie_date, movie_name, current_url]
    print(critic_rating_val, critic_rating_num, user_rating, movie_date, movie_name, current_url)

In [95]:
example_movies = ['/movie/the-sculptress', '/movie/from-justin-to-kelly', '/movie/glitter', '/movie/dungeons-dragons', '/movie/don-peyote', '/movie/the-in-crowd', '/movie/cabin-fever-2016', '/movie/love-wedding-marriage', '/movie/daddy-day-camp', '/movie/fascination', '/movie/fair-game', '/movie/freddy-got-fingered', '/movie/a-beautiful-life', '/movie/paul-blart-mall-cop-2', '/movie/down-to-you', '/movie/new-best-friend', '/movie/avengers', '/movie/nothing-left-to-fear', '/movie/strange-wilderness', '/movie/i-will-follow-you-into-the-dark-2013', '/movie/cocktail', '/movie/left-behind-2014', '/movie/slackers', '/movie/the-adventures-of-pluto-nash', '/movie/the-master-of-disguise', '/movie/kings-ransom', '/movie/persecuted', '/movie/3-strikes', '/movie/mortal-kombat-annihilation', '/movie/date-movie', '/movie/pinocchio', '/movie/scary-movie-5', '/movie/how-to-make-love-like-an-englishman', '/movie/whipped', '/movie/born-to-be-a-star', '/movie/battlefield-earth-a-saga-of-the-year-3000', '/movie/the-tortured-1969', '/movie/superbabies-baby-geniuses-2', '/movie/alone-in-the-dark', '/movie/atlas-shrugged-iii-who-is-john-galt', '/movie/meet-the-spartans', '/movie/dirty-love', '/movie/state-property', '/movie/among-ravens', '/movie/septic-man', '/movie/transylmania', '/movie/miss-march', '/movie/screwed', '/movie/the-hottie-the-nottie', '/movie/baby-geniuses', '/movie/national-lampoons-gold-diggers', '/movie/the-human-centipede-iii-final-sequence', '/movie/vulgar', '/movie/strippers', '/movie/hillarys-america-the-secret-history-of-the-democratic-party', '/movie/the-singing-forest', '/movie/united-passions', '/movie/bio-dome', '/movie/chaos']
#example_movies = ['/movie/the-sculptress', '/movie/from-justin-to-kelly']
len(example_movies)

59

In [96]:
def movie_extraction(movie_list):
    for url in movie_list:
        full_url = 'http://www.metacritic.com'+url
        soup = read_page(full_url)
        ratings = extract_movie_deets(soup, url)
        print(ratings)

In [97]:
results = movie_extraction(example_movies)

['14', '8', '4.8', 'Oct 27, 2000', 'The Sculptress', '/movie/the-sculptress']
['14', '16', '2.8', 'Jun 20, 2003', 'From Justin to Kelly', '/movie/from-justin-to-kelly']
['14', '23', '4.2', 'Sep 21, 2001', 'Glitter', '/movie/glitter']
['14', '25', '4.7', 'Dec 8, 2000', 'Dungeons & Dragons', '/movie/dungeons-dragons']
['14', '8', '3.0', 'May 16, 2014', 'Don Peyote', '/movie/don-peyote']
['14', '16', '1.0', 'Jul 19, 2000', 'The In Crowd', '/movie/the-in-crowd']
['14', '7', '2.0', 'Feb 12, 2016', 'Cabin Fever', '/movie/cabin-fever-2016']
['13', '10', '3.1', 'Jun 3, 2011', 'Love, Wedding, Marriage', '/movie/love-wedding-marriage']
['13', '19', '1.9', 'Aug 8, 2007', 'Daddy Day Camp', '/movie/daddy-day-camp']
['13', '9', '3.6', 'Jan 28, 2005', 'Fascination', '/movie/fascination']
['13', '18', '4.6', 'Nov 3, 1995', 'Fair Game', '/movie/fair-game']
['13', '25', '5.9', 'Apr 20, 2001', 'Freddy Got Fingered', '/movie/freddy-got-fingered']
['13', '7', '1.8', 'Oct 2, 2009', 'A Beautiful Life', '/mov

In [76]:
#tests
#soup = read_page('http://www.metacritic.com/movie/the-sculptress')
#extract_movie_deets(soup, '/movie/the-sculptress')